In [106]:
from pprint import pprint

import yaml
import mock_parser
from sympy import (
    Eq,
    Symbol,
    diff,
    init_printing,
    simplify,
    solve,
)
from sympy.parsing.sympy_parser import parse_expr
from sympy.printing.latex import LatexPrinter


# Define your own derivative notation
def my_derivative_notation(self, expr):
    function = expr.args[0]
    variable = expr.args[1][0]  # Only get the variable, not the order
    return "\partial_{%s} %s(%s, %s)" % (
        self._print(variable),
        self._print(function.func),
        self._print(function.args[0]),
        self._print(function.args[1]),
    )


# Add your notation to the printer
LatexPrinter._print_Derivative = my_derivative_notation


# Use init_printing to apply changes globally
init_printing(use_latex="mathjax")

## Load file contents


In [107]:
with open("ret_pen.yml", "r") as config:
    model_config = yaml.safe_load_all(config)

    model = [stage for stage in model_config]

## Model metadata


In [108]:
print(model[0]["model"])
print(model[0]["author"])
print(model[0]["affiliation"])
print(model[0]["date"])
pprint(model[0]["abstract"])

Consumption-Pension Deposit Model
Alan Lujan
['Johns Hopkins University', 'Econ-ARK']
12/13/2023
('This model is a lifecycle model with two life stages: working life and '
 'retirement. The working life stage is further divided into three blocks: '
 'expectation, deposit, and consumption. During the working life stage, a '
 'consumer receives permanent and transitory income shocks and  decides how '
 'much to consume and how much to deposit in a pension fund. The pension fund '
 'is invested in a risky asset and the consumer receives a risky return on the '
 'pension fund. The consumer also has a liquid asset which is invested in a '
 'risk-free account.\n')


# Stage 1: Working Life before Retirement


In [109]:
print(model[1]["name"])
pprint(model[1]["description"])

working
'working life before retirement'


In [110]:
model[1]["states"]

[{'name': 'aNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (a)ssets after consumption'},
 {'name': 'bNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (b)alance in pension fund after deposit'},
 {'name': 'jNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized wealth (j)ust before income'},
 {'name': 'kNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized pension (k)apital before income'},
 {'name': 'lNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (l)iquid assets after deposit'},
 {'name': 'mNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (m)arket resources'},
 {'name': 'nNrm',
  'domain': [0.0, 'inf'],
  'description': 'normalized (n)et retirement balance'}]

In [111]:
model[1]["controls"]

[{'name': 'cNrm',
  'domain': [0.0, 'lNrm'],
  'description': 'normalized consumption'},
 {'name': 'dNrm',
  'domain': [0.0, 'mNrm'],
  'description': 'normalized pension deposit'}]

In [112]:
model[1]["distributions"]

[{'name': 'perm',
  'description': 'permanent income shock',
  'distribution': <mock_parser.MeanOneLogNormal at 0x1d542569660>},
 {'name': 'tran',
  'description': 'transitory income shock',
  'distribution': <mock_parser.MeanOneLogNormal at 0x1d54256b4f0>},
 {'name': 'risky',
  'description': 'risky asset return',
  'distribution': <mock_parser.LogNormal at 0x1d54256b430>}]

## Stage 1: Blocks


In [113]:
[block["name"] for block in model[1]["blocks"]]

['expectation', 'deposit', 'consumption']

### Expectation Block


In [114]:
model[1]["blocks"][0]

{'name': 'expectation',
 'description': 'expectation block',
 'states': [{'name': 'jNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized wealth (j)ust before income'},
  {'name': 'kNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized pension (k)apital before income'}],
 'distributions': [{'name': 'perm',
   'description': 'permanent income shock',
   'distribution': <mock_parser.MeanOneLogNormal at 0x1d542569660>},
  {'name': 'tran',
   'description': 'transitory income shock',
   'distribution': <mock_parser.MeanOneLogNormal at 0x1d54256b4f0>},
  {'name': 'risky',
   'description': 'risky asset return',
   'distribution': <mock_parser.LogNormal at 0x1d54256b430>}],
 'parameters': ['CRRA',
  'Rfree',
  'PermGroFac',
  'std_perm',
  'std_tran',
  'mean_risky',
  'std_risky'],
 'equations': {'objective': ['v(jNrm, kNrm) = PermAdj^(1-CRRA) * w(mNrm, nNrm)'],
  'transitions': ['mNrm = Rfree * jNrm / PermAdj + tran',
   'nNrm = risky * kNrm / PermAdj'],
  'definitions': 

### Deposit Block


In [115]:
deposit_block = model[1]["blocks"][1]
deposit_block

{'name': 'deposit',
 'description': 'deposit block',
 'states': [{'name': 'mNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized (m)arket resources'},
  {'name': 'nNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized (n)et retirement balance'}],
 'controls': [{'name': 'dNrm',
   'domain': [0.0, 'mNrm'],
   'description': 'normalized pension deposit'}],
 'parameters': ['chi'],
 'equations': {'objective': ['v(mNrm, nNrm) = w(lNrm, bNrm)'],
  'transitions': ['lNrm = mNrm - dNrm', 'bNrm = nNrm + dNrm + g(dNrm)'],
  'definitions': ['g(dNrm) = chi * log(dNrm + 1)']}}

### Consumption Block


In [116]:
consumption_block = model[1]["blocks"][2]
consumption_block

{'name': 'consumption',
 'description': 'consumption block',
 'states': [{'name': 'lNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized (l)iquid assets after deposit'},
  {'name': 'bNrm',
   'domain': [0.0, 'inf'],
   'description': 'normalized (b)alance in pension fund after deposit'}],
 'controls': [{'name': 'cNrm',
   'domain': [0.0, 'lNrm'],
   'description': 'normalized consumption'}],
 'parameters': ['CRRA', 'DiscFac', 'LivPrb'],
 'equations': {'objective': ['v(lNrm, bNrm) = u(cNrm) + beta * w(aNrm, bNrm)'],
  'transitions': ['aNrm = lNrm - cNrm'],
  'definitions': ['u(cNrm) = cNrm ** (1 - CRRA) / (1 - CRRA)',
   'beta = DiscFac * LivPrb']}}

## Endogenous Grid Method


### Deposit Solution


In [117]:
# Parse equations from the dictionary


def parse_eq(eq_str):
    # Remove 'Nrm' from the equation string
    eq_str = eq_str.replace("Nrm", "")

    beta = Symbol("beta")
    rho = Symbol("rho")
    syms = {"beta": beta, "CRRA": rho}

    lhs_str, rhs_str = eq_str.split("=")
    lhs = parse_expr(lhs_str.strip(), local_dict=syms)
    rhs = parse_expr(rhs_str.strip(), local_dict=syms)
    return Eq(lhs, rhs)


def parse_block(block):
    objective = parse_eq(block["equations"]["objective"][0])
    transitions = [parse_eq(eq) for eq in block["equations"]["transitions"]]
    definitions = [parse_eq(eq) for eq in block["equations"]["definitions"]]

    return objective, transitions, definitions


def subs_eq_lr(eq, substitutes):
    eq = eq.doit()
    # Substitute transitions and definitions into objective
    for subs_eq in substitutes:
        eq = eq.subs(subs_eq.lhs, subs_eq.rhs)

    return eq.doit()


def subs_eq_rl(eq, substitutes):
    eq = eq.doit()
    # Substitute transitions and definitions into objective
    for subs_eq in substitutes:
        eq = eq.subs(subs_eq.rhs, subs_eq.lhs)

    return eq.doit()

In [118]:
objective_eq, transitions_eqs, definitions_eqs = parse_block(deposit_block)
objective_eq


v(m, n) = w(l, b)

In [119]:
objective_eq = subs_eq_lr(objective_eq, transitions_eqs)
objective_eq = subs_eq_lr(objective_eq, definitions_eqs)

# Simplify the final equation
final_eq = simplify(objective_eq)

final_eq

v(m, n) = w(-d + m, χ⋅log(d + 1) + d + n)

#### Envelope Conditions


In [120]:
ec_m = Eq(diff(final_eq.lhs, "m"), diff(final_eq.rhs, "m"))
ec_m = subs_eq_rl(ec_m, definitions_eqs)
ec_m = subs_eq_rl(ec_m, transitions_eqs)
ec_m

∂             ∂          
──(v(m, n)) = ──(w(l, b))
∂m            ∂l         

In [121]:
ec_n = Eq(diff(final_eq.lhs, "n"), diff(final_eq.rhs, "n"))
ec_n = subs_eq_rl(ec_n, definitions_eqs)
ec_n = subs_eq_rl(ec_n, transitions_eqs)
ec_n

∂             ∂          
──(v(m, n)) = ──(w(l, b))
∂n            ∂b         

#### Euler Equation


In [122]:
euler_eq = Eq(0, diff(final_eq.rhs, "d"))
euler_eq

    ⎛  χ      ⎞ ⎛ ∂                ⎞│                          ⎛ ∂            
0 = ⎜───── + 1⎟⋅⎜───(w(-d + m, ξ₂))⎟│                        - ⎜───(w(ξ₁, χ⋅lo
    ⎝d + 1    ⎠ ⎝∂ξ₂               ⎠│ξ₂=χ⋅log(d + 1) + d + n   ⎝∂ξ₁           

                  ⎞│         
g(d + 1) + d + n))⎟│         
                  ⎠│ξ₁=-d + m

In [123]:
egm = solve(euler_eq, "d")
solution = egm[0]
solution

  ⎛ ∂                ⎞│                          ⎛ ∂                          
χ⋅⎜───(w(-d + m, ξ₂))⎟│                        - ⎜───(w(ξ₁, χ⋅log(d + 1) + d +
  ⎝∂ξ₂               ⎠│ξ₂=χ⋅log(d + 1) + d + n   ⎝∂ξ₁                         
──────────────────────────────────────────────────────────────────────────────
                        ⎛ ∂                              ⎞│            ⎛ ∂    
                        ⎜───(w(ξ₁, χ⋅log(d + 1) + d + n))⎟│          - ⎜───(w(
                        ⎝∂ξ₁                             ⎠│ξ₁=-d + m   ⎝∂ξ₂   

    ⎞│            ⎛ ∂                ⎞│                       
 n))⎟│          + ⎜───(w(-d + m, ξ₂))⎟│                       
    ⎠│ξ₁=-d + m   ⎝∂ξ₂               ⎠│ξ₂=χ⋅log(d + 1) + d + n
──────────────────────────────────────────────────────────────
            ⎞│                                                
-d + m, ξ₂))⎟│                                                
            ⎠│ξ₂=χ⋅log(d + 1) + d + n                         

In [124]:
# needs to be run twice for some reason

solution = subs_eq_rl(solution, definitions_eqs)
solution = subs_eq_rl(solution, transitions_eqs)

#### Inverted Euler Equation


In [125]:
Eq(Symbol("\mathfrak{d}"), simplify(solution))

                   ∂             ∂             ∂          
               - χ⋅──(w(l, b)) - ──(w(l, b)) + ──(w(l, b))
                   ∂b            ∂b            ∂l         
\mathfrak{d} = ───────────────────────────────────────────
                        ∂             ∂                   
                        ──(w(l, b)) - ──(w(l, b))         
                        ∂b            ∂l                  

### Consumption Solution


In [126]:
objective_eq, transitions_eqs, definitions_eqs = parse_block(consumption_block)
objective_eq


v(l, b) = β⋅w(a, b) + u(c)

In [127]:
objective_eq = subs_eq_lr(objective_eq, transitions_eqs)
objective_eq = subs_eq_lr(objective_eq, definitions_eqs)

# Simplify the final equation
final_eq = simplify(objective_eq)

final_eq

                                                 1 - ρ
          DiscFac⋅LivPrb⋅(ρ - 1)⋅w(-c + l, b) - c     
v(l, b) = ────────────────────────────────────────────
                             ρ - 1                    

#### Envelope Conditions


In [128]:
ec_l = Eq(diff(final_eq.lhs, "l"), diff(final_eq.rhs, "l"))
ec_l = subs_eq_rl(ec_l, definitions_eqs)
ec_l = subs_eq_rl(ec_l, transitions_eqs)
ec_l

∂               ∂          
──(v(l, b)) = β⋅──(w(a, b))
∂l              ∂a         

In [129]:
ec_b = Eq(diff(final_eq.lhs, "b"), diff(final_eq.rhs, "b"))
ec_b = subs_eq_rl(ec_b, definitions_eqs)
ec_b = subs_eq_rl(ec_b, transitions_eqs)
ec_b

∂               ∂          
──(v(l, b)) = β⋅──(w(a, b))
∂b              ∂b         

#### Euler Equation


In [130]:
euler_eq = Eq(0, diff(final_eq.rhs, "c"))
euler_eq

                                                          1 - ρ        
                             ⎛ ∂           ⎞│            c     ⋅(1 - ρ)
    - DiscFac⋅LivPrb⋅(ρ - 1)⋅⎜───(w(ξ₁, b))⎟│          - ──────────────
                             ⎝∂ξ₁          ⎠│ξ₁=-c + l         c       
0 = ───────────────────────────────────────────────────────────────────
                                   ρ - 1                               

In [131]:
egm = solve(euler_eq, "c")
solution = egm[0]
solution

      __________________________________________
     ╱                    1                     
    ╱  ──────────────────────────────────────── 
   ╱                  ⎛ ∂           ⎞│          
ρ ╱    DiscFac⋅LivPrb⋅⎜───(w(ξ₁, b))⎟│          
╲╱                    ⎝∂ξ₁          ⎠│ξ₁=-c + l 

In [132]:
# needs to be run twice for some reason

solution = subs_eq_rl(solution, definitions_eqs)
solution = subs_eq_rl(solution, transitions_eqs)

#### Inverted Euler Equation


In [133]:
Eq(Symbol("\mathfrak{c}"), simplify(solution))

                     _______________
                    ╱       1       
\mathfrak{c} =     ╱  ───────────── 
                  ╱     ∂           
               ρ ╱    β⋅──(w(a, b)) 
               ╲╱       ∂a          

In [134]:
solution

      _______________
     ╱       1       
    ╱  ───────────── 
   ╱     ∂           
ρ ╱    β⋅──(w(a, b)) 
╲╱       ∂a          